In [106]:
from IPython.display import HTML
import re
import os
from bs4 import BeautifulSoup

In [108]:
nested_Q = '''
# BEGIN PROB

# BEGIN SUBPROB

This is the description for the first subpart

# BEGIN SOLUTION

and this is the solution for the first subpart

# END SOLUTION

# END SUBPROB

# BEGIN SUBPROB

prob part 2

# BEGIN SOLUTION

sol of part 2: $$f(x^2) \leq [f(x)]^2$$

# END SOLUTION

# END SUBPROB

# END PROB
'''

one_Q = '''
# BEGIN PROB

here's the description of this one problem

# BEGIN SOLUTION

```py
def f(x):
    return 2 * x + 3
```

here is $x$, there is 

$$x^2$$

- list 1
- list 2
- list 3

# END SOLUTION

# END PROB
'''

In [159]:
def pandoc(s, kind='md', flags=''):
    '''Take in a string containing Markdown and return its HTML equivalent, via pandoc'''
    assert kind == 'tex' or kind == 'md', 'kind must be tex or md'

    if not os.path.exists('temp'):
        os.system('mkdir temp')

    in_file = open(f'temp/temp.{kind}', 'w')
    in_file.write(s)
    in_file.close()

    os.system(f'pandoc --metadata title=" " -s temp/temp.{kind} {flags} -o temp/temp.html')

    out_file = open(f'temp/temp.html', 'r')
    out_s = out_file.read()
    os.system('rm -r temp')

    soup = BeautifulSoup(out_s)
    return str(soup.find('body')).replace('<body>', '').replace('</body>', '')

need to factor in question_num

In [160]:
def process_problem(problem_str, problem_num):
    assert problem_str.count('# BEGIN PROB') == problem_str.count('# END PROB') == 1, 'Need exactly one # BEGIN PROB and # END PROB pair'
    
    if '# BEGIN SUBPROB' in problem_str:
        assert problem_str.count('# BEGIN SUBPROB') == problem_str.count('# END SUBPROB'), 'Different number of # BEGIN SUBPROB and # END SUBPROB'
        return process_prob_with_subparts(problem_str, problem_num)
    
    else:
        return process_prob_no_subparts(problem_str, problem_num)

In [161]:
def add_solution_box(solution_str, problem_num):
    '''solution_str must be an HTML containing the solution text.'''
    
    out = f'''<div class="accordion" id="accordionExample">
  <div class="accordion-item">
    <h2 class="accordion-header" id="heading{problem_num}">
      <button class="accordion-button" type="button" data-bs-toggle="collapse" data-bs-target="#collapse{problem_num}" aria-expanded="true" aria-controls="collapse{problem_num}">
        Click to view the solution.
      </button>
    </h2>
    <div id="collapse{problem_num}" class="accordion-collapse collapse collapse" aria-labelledby="heading{problem_num}" data-bs-parent="#accordionExample">
      <div class="accordion-body">
        {solution_str}
      </div>
    </div>
  </div>
</div>'''
    
    return out

In [210]:
def process_problem_no_subparts(problem_str, problem_num, heading='##'):
    '''Used for problems with no subparts, and to process individual subparts'''
    
    # Extract solution    
    if '# BEGIN SOLUTION' in problem_str:
        sep = 'SOLUTION'
    elif '# BEGIN SOLN' in problem_str:
        sep = 'SOLN'
    else:
        raise AssertionError('Neither # BEGIN SOLUTION nor # BEGIN SOLN were found')
        
    exp = f'# BEGIN {sep}([\d\D]*?)# END {sep}'
        
    solution_str = re.findall(exp, problem_str)
    
    if len(solution_str) != 1:
        raise AssertionError('This should not happen')
    
    # Pass solution_str through pandoc first, then give it the solution box
    solution_str_html = pandoc(solution_str[0])
    solution_processed = add_solution_box(solution_str_html, problem_num)
    
    # Get the problem text
    problem_only = re.sub(exp, '', problem_str)
    problem_only = problem_only.replace('# BEGIN PROB', '').replace('# END PROB', '').strip('\n')
    
    # Put it all together
    
    out = f'''
    {heading} Problem {problem_num}
    
    {problem_only}
    
    {solution_processed}
    '''
    
    return out

In [216]:
def process_problem_with_subparts(problem_str, problem_num):
    parts = re.findall(r'# BEGIN SUBPROB([\d\D]*?)# END SUBPROB', problem_str)
    
    out = f'## Problem {problem_num}\n\n'
    
    for i, part in enumerate(parts):
        out += process_problem_no_subparts(part, str(problem_num) + f'.{i+1}', heading='###') + '\n\n'
        
    return out